# Week 5 IBM Data Science Project

### Opening a new Halal restaurant in London, England
#### By: Bilal Siddiqui

#### Step 1) Import libraries

In [70]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [71]:
#! pip install geocoder
import geocoder

#### Step 2) Get URL

In [72]:
data = requests.get("https://www.airbnb.com/locations/london/neighborhoods").text

#### Step 3) Parse data from HTML

In [73]:
soup = BeautifulSoup(data, 'html.parser')

#### Step 4) Create empty list for data

In [74]:
neighborhoodList = []

#### Step 5) Find data

In [75]:
info = soup.find_all('div', class_ = "neighborhood-list section section-offset")

#### Step 6) Get length of data

In [76]:
len(info)

1

#### Step 7) Append data into empty list

In [77]:
for row in soup.find_all("div", class_="neighborhood-list section section-offset")[0].findAll("li"):
    neighborhoodList.append(row.text)

#### Step 8) From list, create dataframe

In [78]:
# create a new DataFrame from the list
df_neigh = pd.DataFrame({"Neighborhood": neighborhoodList})

df_neigh.head()

,Neighborhood
0,Balham
1,Battersea
2,Belgravia
3,Bermondsey
4,Bethnal Green


#### Step 9) Get shape of dataframe

In [79]:
df_neigh.shape

(48, 1)

#### Step 10) Get coordinates

In [80]:
def get_ll(neighborhood):
    l_l_coords = None
    while(l_l_coords is None):
        g = geocoder.arcgis('{}, London, England'.format(neighborhood))
        l_l_coords = g.latlng
    return l_l_coords

#### Step 11) Set coordinates to variable

In [81]:
coords = [get_ll(neighborhood) for neighborhood in df_neigh["Neighborhood"].tolist()]

In [82]:
coords

[[51.44822000000005, -0.1483899999999494],
 [51.46760000000006, -0.16289999999997917],
 [51.49705000000006, -0.15274999999996908],
 [51.49790000000007, -0.08143999999992957],
 [51.52669000000003, -0.06256999999993695],
 [51.520740000000046, -0.12309999999996535],
 [51.45238000000006, -0.1219499999999698],
 [51.54057000000006, -0.1433399999999665],
 [51.54868000000005, -0.0917499999999336],
 [51.49014000000005, -0.16247999999995955],
 [51.52050000000003, -0.09742999999997437],
 [51.46095000000008, -0.13921999999996615],
 [51.522164999999994, -0.10715175000000866],
 [51.51544000000007, -0.1269299999999589],
 [51.49187000000006, -0.19727999999997792],
 [51.51873000000006, -0.1373699999999758],
 [51.47772000000003, -0.2014499999999657],
 [51.48454000000004, 0.002750000000048658],
 [51.53849000000008, -0.07554999999996426],
 [51.49617000000006, -0.22934999999995398],
 [51.48721000000006, -0.013809999999978118],
 [51.532790000000034, -0.1061399999999253],
 [51.487010000000055, -0.11311999999

#### Step 12) Create dataframe for coordinates

In [83]:
df_ll = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

#### Step 13) Merge dataframes

In [84]:
df_neigh['Latitude'] = df_ll['Latitude']
df_neigh['Longitude'] = df_ll['Longitude']

In [85]:
df_neigh

,Neighborhood,Latitude,Longitude
0,Balham,51.448220,-0.148390
1,Battersea,51.467600,-0.162900
2,Belgravia,51.497050,-0.152750
3,Bermondsey,51.497900,-0.081440
4,Bethnal Green,51.526690,-0.062570
5,Bloomsbury,51.520740,-0.123100
6,Brixton,51.452380,-0.121950
7,Camden Town,51.540570,-0.143340
8,Canonbury,51.548680,-0.091750
9,Chelsea,51.490140,-0.162480


#### Step 14) Get shape of new dataframe

In [86]:
df_neigh.shape

(48, 3)

#### Step 15) Save dataframe as CSV

In [87]:
# save the DataFrame as CSV file
df_neigh.to_csv("df_neigh.csv", index=False)

#### Step 16) Get coordinates of London, England using geolocator

In [88]:
address = 'London, England'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London, England are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinates of London, England are 51.5073219, -0.1276474.


#### Step 17) Create map of London

In [89]:
map_le = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_neigh['Latitude'], df_neigh['Longitude'], df_neigh['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_le)  
    
map_le

#### Step 18) Save map as HTML

In [90]:
map_le.save('map_le.html')

#### Step 19) Get FourSquare API credentials

In [91]:
CLIENT_ID = 'E13W3BJRROWTESWAPFEBCTUOXDVUAEQLUCG043YLALFQ4FRS' # your Foursquare ID
CLIENT_SECRET = 'W1WVFOMH4VH3HIJRHCATYFIWVYVXC3SK5DZZ51E14YGSO3MB' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E13W3BJRROWTESWAPFEBCTUOXDVUAEQLUCG043YLALFQ4FRS
CLIENT_SECRET:W1WVFOMH4VH3HIJRHCATYFIWVYVXC3SK5DZZ51E14YGSO3MB


#### Step 20) Get vanue data

In [93]:
radius = 3000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_neigh['Latitude'], df_neigh['Longitude'], df_neigh['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

#### Step 21) Convert venue data into dataframe

In [94]:
venue_df = pd.DataFrame(venues)

venue_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venue_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Balham,51.44822,-0.14839,Ciullosteria,51.447144,-0.148981,Italian Restaurant
1,Balham,51.44822,-0.14839,We Brought Beer,51.444324,-0.150656,Beer Store
2,Balham,51.44822,-0.14839,M1LK,51.444450,-0.150913,Coffee Shop
3,Balham,51.44822,-0.14839,Brickwood Coffee & Bread,51.444509,-0.151127,Coffee Shop
4,Balham,51.44822,-0.14839,Franco Manca,51.443616,-0.149959,Pizza Place


#### Step 22) Print list of categories

In [95]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:100]

array(['Italian Restaurant', 'Beer Store', 'Coffee Shop', 'Pizza Place',
       'Pub', 'Café', 'Butcher', 'Park', 'Steakhouse', 'Breakfast Spot',
       'Restaurant', 'Caucasian Restaurant', 'Wine Shop', 'Fish Market',
       'Gastropub', 'French Restaurant', 'Bakery', 'Fish & Chips Shop',
       'Shop & Service', 'Japanese Restaurant', 'Deli / Bodega',
       'Karaoke Bar', 'Moroccan Restaurant', 'Supermarket',
       'Asian Restaurant', 'BBQ Joint', 'Brewery', 'Plaza',
       'Middle Eastern Restaurant', 'Movie Theater', 'Historic Site',
       'Ice Cream Shop', 'Pakistani Restaurant', 'Bistro',
       'English Restaurant', 'Grocery Store', 'Beer Bar', 'Wine Bar',
       'Market', 'Skate Park', 'Burger Joint', 'Tapas Restaurant',
       'Indian Restaurant', 'Chinese Restaurant', 'Bar', 'Windmill',
       'Farmers Market', 'Caribbean Restaurant', 'Gym / Fitness Center',
       'Seafood Restaurant', 'Theater', 'Thai Restaurant',
       'Indonesian Restaurant', 'Garden Center', 'Vietnam

#### Step 23) Create another dataframe for venues for each neighborhood

In [96]:
df_rando = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

df_rando['Neighborhoods'] = venues_df['Neighborhood'] 

col = [df_rando.columns[-1]] + list(df_rando.columns[:-1])
df_rando = df_rando[col]

df_rando.head()

,Neighborhoods,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distillery,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Observatory,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Balham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Step 24) Merge dataframes

In [97]:
le_merged = df_rando.groupby(["Neighborhoods"]).mean().reset_index()

le_merged

,Neighborhoods,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distillery,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Observatory,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Balham,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00000,

#### Step 25) Length of number of Indian Restaurants

In [98]:
len(le_merged[le_merged["Indian Restaurant"] > 0])

40

#### Step 26) Create new dataframe on neighborhoods and indian restaurants

In [99]:
le_rest = le_merged[["Neighborhoods","Indian Restaurant",]]

In [100]:
le_rest.head()

,Neighborhoods,Indian Restaurant
0,Balham,0.01
1,Battersea,0.01
2,Belgravia,0.03
3,Bermondsey,0.00
4,Bethnal Green,0.02


#### Step 27) Cluster neighborhoods

In [101]:
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

kmeans.labels_[0:10]

array([0, 0, 1, 0, 2, 0, 2, 0, 0, 2], dtype=int32)

#### Step 28) Dataframe for clusters

In [102]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
le_update = le_rest.copy()

# add clustering labels
le_update["Cluster Labels"] = kmeans.labels_

In [103]:
le_update.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
le_update.head()

,Neighborhood,Indian Restaurant,Cluster Labels
0,Balham,0.01,0
1,Battersea,0.01,0
2,Belgravia,0.03,1
3,Bermondsey,0.00,0
4,Bethnal Green,0.02,2


#### Step 29) Merge dataframes again

In [104]:
le_update = le_update.join(df_neigh.set_index("Neighborhood"), on="Neighborhood")

le_update.head()

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Balham,0.01,0,51.44822,-0.14839
1,Battersea,0.01,0,51.46760,-0.16290
2,Belgravia,0.03,1,51.49705,-0.15275
3,Bermondsey,0.00,0,51.49790,-0.08144
4,Bethnal Green,0.02,2,51.52669,-0.06257


#### Step 30) Sort by cluster labels

In [105]:
le_update.sort_values(["Cluster Labels"], inplace=True)
le_update

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Balham,0.010000,0,51.448220,-0.148390
28,Marylebone,0.000000,0,51.523840,-0.157570
24,Kings Cross,0.010000,0,51.530890,-0.120800
22,Kennington,0.010000,0,51.487010,-0.113120
30,Notting Hill,0.010000,0,51.512440,-0.206390
31,Paddington,0.010000,0,51.515640,-0.181080
18,Haggerston,0.010000,0,51.538490,-0.075550
17,Greenwich,0.010000,0,51.484540,0.002750
33,Shepherd's Bush,0.010000,0,51.505290,-0.217910
26,Lambeth,0.000000,0,51.490840,-0.111080


#### Step 31) Create clustering map 

In [106]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(le_update['Latitude'], le_update['Longitude'], le_update['Neighborhood'], le_update['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Step 32) Save cluster map as HTML

In [107]:
map_clusters.save('map_clusters.html')

#### Step 33) Cluster label 0

In [108]:
le_update.loc[le_update['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Balham,0.01,0,51.448220,-0.148390
28,Marylebone,0.00,0,51.523840,-0.157570
24,Kings Cross,0.01,0,51.530890,-0.120800
22,Kennington,0.01,0,51.487010,-0.113120
30,Notting Hill,0.01,0,51.512440,-0.206390
31,Paddington,0.01,0,51.515640,-0.181080
18,Haggerston,0.01,0,51.538490,-0.075550
17,Greenwich,0.01,0,51.484540,0.002750
33,Shepherd's Bush,0.01,0,51.505290,-0.217910
26,Lambeth,0.00,0,51.490840,-0.111080


#### Step 34) Cluster label 1

In [109]:
le_update.loc[le_update['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
39,St. James's,0.040000,1,51.507820,-0.133560
35,Soho,0.030000,1,51.511830,-0.132550
34,Shoreditch,0.030000,1,51.524960,-0.081410
42,Stockwell,0.040000,1,51.470010,-0.112490
32,Pimlico,0.040000,1,51.490800,-0.139090
29,Mayfair,0.040000,1,51.508680,-0.143890
45,Westminster,0.030000,1,51.500080,-0.128020
37,South Bank,0.047619,1,51.428664,0.069842
23,Kensington,0.030000,1,51.499060,-0.198740
46,Whitechapel/Brick Lane,0.030000,1,51.521037,-0.071833


#### Step 35) Cluster label 2

In [110]:
le_update.loc[le_update['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
16,Fulham,0.02,2,51.477720,-0.201450
36,Somers Town,0.02,2,51.535886,-0.127526
20,Isle of Dogs,0.02,2,51.487210,-0.013810
9,Chelsea,0.02,2,51.490140,-0.162480
40,St. Luke's,0.02,2,51.488160,-0.189260
41,Stepney,0.02,2,51.512436,-0.039150
6,Brixton,0.02,2,51.452380,-0.121950
43,The West End,0.02,2,51.514140,-0.155100
4,Bethnal Green,0.02,2,51.526690,-0.062570
21,Islington,0.02,2,51.532790,-0.106140
